# **Garden Optimization Problems**

![title](images/cartoon.png)


## **Motivation**
Over the course of this project we will develop a real life application for D-Wave by creating our own QUBO problem from scratch through guided exercises, and later on make some basic problem submissions to D-Wave devices.

What we will focus on: 
- recognizing a real-life discrete optimization problem which is suitable for solving on a quantum annealer,
- learning how to state this problem mathematically, 
- casting this formulation into a D-Wave-compatible QUBO form,
- submitting this problem to D-Wave's LeapHybridSolver and analyzing the results.

What we will *not* focus on:
- advanced usage of the bare QPUs
- embedding of problems to target specific QPUs,
- fine-tuning of parameters like the chain-strength, annealing time, reverse annealing schemes,...


## **Introduction**

For this problem, the objective is to find an optimal placement of vegetable plants in a garden, respecting that some plant species have friendly, neutral, or antagonistic relations with other species (see Fig. 1, where -1 = friendly, 0 = neutral and +1 = antagonistic relationship), a technique known as companion planting. For instance, tomato and lettuce have a friendly relationship and could be placed next to each other, whereas tomato and cucumber have an antagonistic relationship and should be placed apart from each other.

![title](images/companions.svg)


# **Section 1:** Setting up

In this first section we will create the inputs required to generate our QUBO problem in later sections. 

## **Exercise 1.1:** Creating the garden

As a first step we need to create the garden by laying out the pots where we will place our plants in later stages.

**a)** Create a *graph* $G$ of the garden pots arranged in a square grid with $k$ columns and $l$ rows, where nodes represent pots and edges between nodes are only present if the pots are adjacent. 

**HINT:** `networkx` has a lot of [graph generators](https://networkx.org/documentation/networkx-2.6.2/reference/generators.html).



**b)** Create a sorted list of pots $n$.



**c)** Plot the garden graph. Make sure that the pot names are visible in the plot, and adjacent pots are connected by edges. Here is an example of how such plot could look like: 

![title](images/garden_grid.png)



**d)** Create an adjacency matrix $J$ of size $(n,n)$ for the garden, where
$$
J_{ii'} = \begin{cases}
        1 & \text{if $i<i'$ and pot $i$ and $i'$ are adjacent} \\
        0 & \text{otherwise}
    \end{cases}
$$




In [5]:
%matplotlib inline

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt


## **Exercise 1.2:** Loading the companion species table

Within this directory you will find a [CSV](https://en.wikipedia.org/wiki/Comma-separated_values) file `./companions.csv`. This file contains the relationships between different plant species which we may choose to plant in our garden, where $-1$ indicates a friendly relationship, $0$ is neutral and $-1$ is antagonistic.

**a)** Import and visualize this file.

**b)** Create a sorted list of plant species $p$.

**c)** Create a matrix $C$ of plant species relationships indexed by $p$.

In [6]:
import pandas as pd
import numpy as np

filename = './companions.csv'


## **Exercise 1.3:** Deciding what plants we want in the garden

In this exercise we will figure out what plants will be planted in the $n$ pots of our garden.

**a)** Write a function which randomly samples $n$ times from the set of plants defined in $p$ and returns a list of plants $t$. This function should be able to perform sampling with or without replacement.

**b)** Modify the function from (a) to return a histogram in the form of a tuple $(t,c)$ where:
- $t$ is the list of *unique* plants sampled and 
- $c$ is the number of times each element of $t$ was sampled.

**c)** Verify that the function in (b) returns as many plants as there are pots in our garden.

# **Section 2**: Creating the cost function

Now that we have generated the input variables which we needed to determine the layout of our garden, the plants we want to plant and their interrelationships, we can move on to formulating the problem.

D-Wave Quantum Annealers require that the optimisation problem is submitted in [QUBO](https://en.wikipedia.org/wiki/Quadratic_unconstrained_binary_optimization) format (or the physics equivalent Ising model). QUBO/Ising have in common that the problem variables over which the cost function of our problem is formulated are *binary* ($\{0,1\}$ for QUBO or $\{-1,+1\}$ for Ising).

Given the pots in the garden numbered by $i = 0 .. n-1$ and the selected plant species numbered by $j = 0 .. t-1$, let us define the *binary* decision variables vector for our QUBO problem to be:

$$
\mathbf{x} = (x_{0,0}, x_{0,1}, \ldots, x_{{n-1},{t-1}})
$$

where each entry of vector $\mathbf{x}$ takes the following value:

$$
x_{ij} = \begin{cases}
        1 & \text{if plant $j$ should be planted in pot $i$ } \\
        0 & \text{otherwise}
    \end{cases}
$$

**QUESTION:** How many bits does vector $\mathbf{x}$ have?

Using this encoding, the solution to our optimization problem should be an assignment of $x_{ij}$ values which:
1) Fills each of our pots $i$ in $n$ with exactly one of the plants listed in $t$,
2) Plants each plant $j$ in our list $t$ exactly $c_j$ times,
3) Minimizes the unhappiness of all plants due to each being planted next to an incompatible neighbor.

**QUESTION:** Assuming that constraints (1) and (2) are respected, how many entries of $\mathbf{x}$ should take value $1$?

Over the exercises in this section we will create mathematical expression of a QUBO problem which ensures that constraints (1) and (2) are respected and returns a minimum energy for an optimal arrangement of plants according to (3) is found, and higher energies for less suitable arrangements. 


**HINT:** You can write math notation in a Jupyter Notebook by creating a Markdown cell and writing your LaTeX math code between `$$ $$` symbols.

## **Exercise 2.1:** Encoding and decoding between problem and qubit spaces

**a)** Write an `encode()` function which maps every possible $(i,j)$ pot-plant combination to an integer between $0..(n\dot{t}-1)$.

**b)** Write the reverse `decode()` function which maps an integer between $0..(n\dot{t}-1)$ to one of the possible $(i,j)$ pot-plant combinations.

**c)** Verify that `i,j = decode(encode(i,j))` for all $i \in n$ and $j \in t$.

In [7]:
#a)

def encode():
    return index


#b)

def decode():    
    return pot,species

## **Exercise 2.2:** Fill every pot

In this exercise we will build the expression for constraint (1).

**a)** Design a mathematical expression $k_i(\mathbf{x})$ which returns a value of $0$ if exactly one plant $j$ is planted in pot $i$, and larger values for other assignments of $\mathbf{x}$ which do not respect the constraint.

**b)** Build upon $k_i(\mathbf{x})$ to design a mathematical expression $k(\mathbf{x})$ which returns a value of $0$ if exactly one plant $j$ is planted in pot $i$ for each pot.

**c)** Expand out any higher order terms in $k(\mathbf{x})$ and extract any constant terms out of the summations.

**HINT:** Remember that $x_{ij}$ is binary, which means that the statement $x_{ij}^2=x_{ij}$ holds true for whichever value $x_{ij}$ takes. 

## **Exercise 2.3:** Plant every plant

In this exercise we will build the expression for constraint (2).

**a)** Design a mathematical expression $l_j(\mathbf{x})$ which returns a value of $0$ if plant $j$ in $t$ has been planted in the garden exactly $c_j$ times, and larger values for assignments of $\mathbf{x}$ which do not respect the constraint.

**b)** Build upon $l_j(\mathbf{x})$ to design a mathematical expression $l(\mathbf{x})$ which returns a value of $0$ if plant $j$ in $t$ has been planted in the garden exactly $c_j$ times for each plant type.

**c)** Expand out any higher order terms in $l(\mathbf{x})$ and extract any constant terms out of the summations.

## **Exercise 2.4:** Minimize plant unhappiness

In this exercise we will build the expression for the cost function in (3). 

**a)** Starting with a garden of just two neighboring pots $i$ and $i'$ and two plant species $j$ and $j'$, design a mathematical expression $E(x_{ij},x_{i'j'})$ which takes the following values:

$$
E(x_{ij},x_{i'j'}) = \begin{cases}
                        0 & \text{if species $j$ and $j'$ have a friendly relationship.} \\
                        1 & \text{if species $j$ and $j'$ have a neutral relationship.} \\
                        2 & \text{if species $j$ and $j'$ have an antagonistic relationship.}
                      \end{cases}
$$

**b)** Build upon (a) to create a mathematical expression $E(\mathbf{x})$ which iterates over all $(i,i')$ pairs of pots in a garden with $n$ (instead of just $2$) pots, and adds up the contributions of their respective $E(x_{ij},x_{i'j'})$.

**c)** Modify the expression in (b) so that only plants in adjacent pots contribute to $E(\mathbf{x})$.

**HINT:** Remember the adjacency matrix $J$ from Ex. 1.1?

**d)** Expand out any higher order terms in $E(\mathbf{x})$ and extract any constant terms out of the summations.

# **Section 3**: Creating and submitting the QUBO problem

In Section 2 we have created the mathematical expressions that describe our *Optimization* problem, as well as some constraints which need to be respected by a valid solution, but we don't have a QUBO formulation yet. Can you see what we are still missing?

These expressions are formulated over *binary* variables $x_{ij}$, which means that we already have a *Binary Optimization* problem.

The meaning of *quadratic* in the QUBO model is that the highest-order interaction allowed between problem variables is quadratic (i.e. order 2), or in other words there is no term in the cost function which involves a product of 3 or more $x_{ij}$ variables.

**QUESTION:** Have a look at the expressions produced in exercises 2.2-2.4, do you see any term which is neither linear nor quadratic? De we have a *Quadratic Binary Optimization* problem so far?

Seems like we are almost there, but there's still these annoying constraints from exercises 2.2 and 2.3 standing on our way to a *Quadratic Unconstrained Binary Optimization* problem. How do we get rid of the constraints while still ensuring that these are respected by our final QUBO?

The answer to this is to introduce *penalty parameters* which add a large contribution to the final energy of the solution, only if a constraint has been violated. Therefore, we want to combine the expressions from ex. 2.2-2.4 as follows:

$$
Q(\mathbf{x}) = E(\mathbf{x}) + \lambda_1 k(\mathbf{x}) + \lambda_2 l(\mathbf{x})
$$

where $\lambda_1$ and $\lambda_2$ are chosen to be sufficiently large. Let us for now choose these to be $\lambda_1 = \lambda_2 = 2$.

## **Exercise 3.1:** Coding the QUBO.

In this exercise we will translate from the mathematical formulation created in section 2 to a function which will return a `dimod.BinaryQuadraticModel` instance. This is a data structure used in `D-Wave Ocean SDK` to store and manipulate QUBO/Ising problems.


**a)** Write a function which builds the QUBO problem instance based on the mathematical formulation of Section 2 and returns a `dimod.BinaryQuadraticModel` object.

We will first create a Python dictionary `Q` and we will add entries to it where the key should be a tuple of integers $(u,v)$ and the value should be the corresponding bias/interaction strength between the $x_u$ and $x_v$ variables.

**HINT:** Use the `encode()` function to map between the problem-domain $(i,j)$ pot-plant pairs and the qubit index $u$ which we will use as key for dictionary `Q`.

**b)** Write a plotting function which displays the QUBO matrix as a heatmap. This kind of visualization is often helpful to get an overview of the density and strength of the interactions between problem variables.

In [8]:
#a)

from dimod import BinaryQuadraticModel

def build_bqm(l1, l2, n, J, p, C, t, c):
    
    # initialize QUBO
    Q = dict()
    offset = 0 # we will add up all the constant terms (i.e. terms which aren't a function of x) in this variable
    
    # If you want to add term {a} at position Q[index1,index2] use the following pattern:
    # index1 = encode(pot1, species1, n, t)
    # index2 = encode(pot2, species2, n, t)
    # Q[(index1,index2)] = Q.get((index1,index2), 0) + a
    
    ### YOUR CODE GOES HERE:
    
    # constraint k(x)
    
    # constraint l(x)

    # objective function E(x)

    ###
    
    Q = {term: Q[term] for term in Q if Q[term] !=0}
    bqm = BinaryQuadraticModel.from_qubo(Q, offset=offset)
    
    return bqm

#b) 

def plot_qubo():
    ### YOUR CODE GOES HERE:
    
    ###
    return

## **Exercise 3.2:** Solving the QUBO and evaluating the results.

**a)** Create a small problem instance with a $(2,2)$ garden grid and use `dimod.ExactSolver()` to solve it. (Here is a link to the [docs](https://docs.ocean.dwavesys.com/en/stable/docs_dimod/index.html#example-usage)).

**HINT:** `ExactSolver` is a brute-force solver, which means that any problem instance bigger than $(2,2)$ will make the solver crash.

**b)** Inspect the resulting `SampleSet` object and extract a solution bitstring corresponding to a ground state of the problem.

**QUESTION:** Can you list some reasons why this is a [degenerate](https://en.wikipedia.org/wiki/Degenerate_energy_levels) problem?

**c)** Using the `decode` function, decypher the (pot,plant) combination suggestions contained in the solution bitstring. 

**d)** Write a function `is_valid` which checks the (pot,plant) choices contained in the bitstring respect the constraints.

**e)** Write a plotting function which displays the (pot,plant) choices of a valid solution so they reflect friendly/neutral/antagonistic relationships between neighboring plants:

![title](images/solution.png)

In [9]:
from dimod import ExactSolver

#a) 


#b)


#c)

def decypher():
    ### YOUR CODE GOES HERE:
    
    ###
    return choices


#d)

from collections import Counter

def is_valid():
    ### YOUR CODE GOES HERE:
    
    ###
    return bool(valid)


#e)

def plot_solution():
    ### YOUR CODE GOES HERE:
    
    ###
    return 

## **Exercise 3.3:** Solving larger QUBO instances with LeapHybridSampler.

For larger problem instances, a relatively simple and powerful solver is [LeapHybridSampler](https://docs.ocean.dwavesys.com/en/stable/docs_system/reference/samplers.html#leaphybridsampler). This solver can admit problems with up to a million binary variables, way beyond the capabilities of bare QPUs.

**a)** Generate a problem instance with $100$ pots and submit it to `LeapHybridSampler`. 

**b)** Inspect the returned sample. Is it a valid solution? If not can you think of a way to encourage the sampler to return valid solutions? If there is, try it and see if the situation improves.

In [10]:
from dwave.system import LeapHybridSampler


# **(Optional) Section 4**: OK I'm done, what now?

We have already covered the basics of designing QUBO problem formulations. In the remaining exercises we will cover more advanced or creative topics, as well as give you pointers to other notebooks in this course if you are interested in specific topics which we haven't touched upon here.

There may not be enough time to do all of these below, so if you got here read through the exercises below and decide if there's any which you want to focus on.

## **Exercise 3.4:** Finding optimal $\lambda_1$ and $\lambda_2$ penalty terms.

How do we decide what *sufficiently large* penalty terms $\lambda_1$ and $\lambda_2$ are? 

In the last exercise you might have manually tuned $\lambda_1$ and $\lambda_2$ until you got valid results. This is a reasonable heuristic method which works well in many scenarios, but are there ways to make more educated guesses regarding the optimal values of the penalty parameters.

Intuitively, we want any valid (i.e. constraint-respecting) solution to have a lower energy than any invalid solution, regardless of how bad this solution is in terms of unfriendliness between neighbors. Therefore, we will need to 
   1. calculate the energy of the worst possible *valid* solution,
   2. calculate the energy of the best possible *invalid* solution,
   3. set $\lambda_1$ and $\lambda_2$ so that the energy of case (2) is slightly larger than that of case (1). 
   
   
For a garden arranged as a square grid of $(q,r)$ dimensions and an arbitrary choice of $qr$ plants for $qr$ pots,

**a)** What's the largest possible value that $E(\mathbf{x})$ could take? 

**b)** What's the smallest possible value that $Q(\mathbf{x}$ could take if a constraint is violated once? (Assume that the assignment is maximally-friendly otherwise).

**c)** Using (a) and (b), find the optimal value for the penalty terms $\lambda_1$ and $\lambda_2$.

**d)** Repeat exercise 3.2 and check if all valid solutions have lower energy than invalid ones.

**e)** Repeat exercise 3.3. Are you getting valid solutions more often now?

## **Exercise 3.5:** Extending the problem

Do you have any ideas to modify the QUBO formulation? Using the basic formulation as a template, get creative and come up with additional constraints, modify the cost function, etc.

Here are a couple of suggestions for inspiration, but of course feel free to implement your own ideas!

- Some of the plant species we deal with here are small (namely Basil, Thyme, Parsley, Chives, Rosemary, Sage, Dill, Coriander and Mint) whereas the others are big. Imagine the bottom row of our garden grid is facing the south and is therefore getting more direct sun. Can you think of a way to modify the QUBO so that all the plants get their fair share of sun, i.e. the tall plants don't shadow the small ones?


- Play around with the arrangement of pots. A 2D rectangular grid of fixed length is way too restrictive.


- ...?

## **Exercise 3.6:** Analyzing and visualizing the results

When solving problems with quantum annealers it is often the case that one needs to repeatedly analyze large volumes of data during the process of implementing the QUBO, tuning parameters and overall attempting to obtain the best results.

In order to do this efficiently it makes sense to create tools which allow you to analyze (or even better visualize) the data you are working with as clearly as possible.

Can you think of any such visualization tools which would be helpful for investigating this QUBO problem?

# Other topics discussed in this course

In this notebook we have covered the basics of formulating a real problem as a QUBO, which is the minimum requirement for submitting a problem to a DWave quantum annealer, and we have submitted this problem to the Leap hybrid solver. Hybrid solvers have a significantly simpler interface than the "bare" DWave QPUs. Therefore, a lot of relevant details about how to tune a QPU to deliver optimal results for your QUBO have been omitted here. 

If you are interested in using the QPU directly you should check out [this notebook](./../tsp_DWAVE/traveling-salesman-problem-dwave.ipynb). In it you will learn about embedding QUBO problems onto the QPU and tuning parameters such as the chain strength and the annealing time. Knowing how to tune these will ensure that you get the best results out of the quantum annealer! The notebook uses the Travelling Salesperson Problem (TSP) as a guide, but many of the tuning strategies discussed there are just as well applicable to other QUBO problems.*

Going beyond quantum annealing, QUBO problems serve as a bridge towards gate-based quantum computing, as these can also be solved on digital quantum computers using a variational method known as the Quantum Approximate Optimization Algorithm (QAOA). You can learn about applying the QAOA algorithm to solve the garden optimization problem using Qiskit and tuning the algorithm to obtain best results in [this notebook](./../garden_QAOA/garden-qaoa.ipynb).

In [this other notebook](./../knapsack_QAOA/knapsack-qaoa.ipynb) you can learn about the Knapsack problem, an NP-complete optimization problem with an important application in the field of logistics. Solving this problem allows you to fit as many packages as possible into a single delivery truck, which could save you lots of money if you happen to own Amazon Inc., for example. The notebook will also teach you how to solve the problem using QAOA and tune the algorithm to obtain optimal results.

(* NOTE: If you are interested in solving garden optimization problems on the QPU make sure to work with instances of at most 10 pots, as these problems are already too large for the QPU).